In [46]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

# Retrograde Analyse für das Schach-Endspiel

Dieses Notebook wird zur Berechnung der $S_n$ Mengen verwendet. Diese werden benötigt, um letztendlich ein Schach-Endspiel
lösen zu können. Die $S_n$ Mengen werden mihilfe der Retrograde Analyse bestimmt.

## Imports

Im Rahmen dieses Notebooks werden für die Darstellung und Durchführung der besagten Analyse folgende drei Bibliotheken verwendet:

* ``chess``: Die Python-Schach-Bibliothek, mit welcher Schachbretter dargestellt werden, Züge und Zustände ausgewertet etc.
* ``clear_output`` aus der IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen
verwendet wird.
* ``json``: Wird zum Speichern der Ergebnisse verwendet.

In [47]:
import chess
from IPython.display import clear_output
import json

## Funktionen zur Bestimmung aller gültigen Positionen

Ein Schachbrett besteht aus insgesamt acht Spalten und Zeilen. Die Spalten werden durch Buchstaben gekennzeichnet, die Zeilen wiederum durch Zahlen. Aus der Kombination einer Spalte (z.B. a) und einer Zahl (z.B. 1) erhält man eine eindeutige
Kennzeichnung für ein Feld (z.B. a1).

Im folgenden Code werden die Buchstaben a bis h mit den Zahlen 1 bis 8 zu diesen Feldnamen konvertiert und in
``all_squares`` gespeichert.

In [48]:
column_convert = {
    1 : 'a',
    2 : 'b',
    3 : 'c',
    4 : 'd',
    5 : 'e',
    6 : 'f',
    7 : 'g',
    8 : 'h'
}

all_squares = []
for row in range(1,9):
    for col_num in range(1,9):
        column = column_convert[col_num]
        all_squares.append(column + str(row))
# print(all_squares)

Da nun soweit alle möglichen Positionen auf dem Schachfeld bestimmt worden sind, müssen nun die Spielfiguren angegeben werden, für die die Endspiel-Datenbank berechnet werden soll. Hierfür werden in der Liste ``figures_to_place`` alle Figuren aufgeführt. Die Einträge der Liste werden als Tupel bestehend aus Figur und Farbe gespeichert.
Bsp.: ``(chess.KING, chess.WHITE)``

Die Reihenfolge oder Position der Figur ist für die Berechnung irrelevant.
Diese wird erst im nächsten Schritt (der Auswertung) benötigt.

In [49]:
figures_to_place = [(chess.ROOK, chess.WHITE), (chess.KING, chess.WHITE), (chess.KING, chess.BLACK)]

Das Erstellen jeglicher Boards wird mit der Funktion ``place_figure_everywhere_on_every_board`` umgesetzt. Diese erhält folgende Parameter: 

* ``figure``: Die zu platzierende Figur, als Objekt der chess-Library
* ``list_of_boards``: Eine Liste mit Boards, auf welchen die Figur platziert werden soll

Die Liste ``list_of_boards`` wird bei dieser Funktion nur per Call by Value aufgerufen, weshalb auch keine Annomalien beim Ausführen der Funktion auftreten. Der hier verwendete Algorithmus kann anhand folgender Punkte nachvollzogen werden:

* Über alle Spielbretter in der ``list_of_boards`` iterieren.
* Die bereits verwendeten Spielfelder speichern.
* Über alle Spielfelder iterieren.
* Wenn das Spielfeld noch nicht benutzt ist:
  * Das Spielbrett kopieren und die Figur auf diesem Feld platzieren.
  * Die Kopie der Ergebnisliste anfügen.

Die Funktion gibt als Ergebnis eine Liste aller Spielbretter gespeichert in ``result_list`` zurück.

In [50]:
def place_figure_everywhere_on_every_board(figure, list_of_boards):
    result_list = []
    for board in list_of_boards:
        cur_piece_map = board.piece_map()
        squares_used = list(cur_piece_map.keys())
        for square in all_squares:
            parsed_square = chess.parse_square(square)
            if parsed_square not in squares_used:
                tmp_board = board.copy()
                tmp_board.set_piece_at(parsed_square, figure)
                result_list.append(tmp_board)
    return result_list

Da aber nicht automatisch alle Spielsituationen, die mit der Funktion ``place_figure_everywhere_on_every_board`` generiert werden, den Schachregeln folgen, wurde eine weitere Hilfsfunktion definiert, die für eine gegebene Liste an Spielbrettern (``board_list``) überprüft, ob diese valide sind. In der Funktion ``check_boards`` wird über jedes Spielbrett der Liste iteriert und nur die Bretter zurückgegeben, die auch wirklich den Schachregeln entsprechen. 

In [51]:
def check_boards(board_list):
    result_list = []
    for board in board_list:
        if board.is_valid():
            result_list.append(board)
    return result_list

## Die Ursprungsmenge S erstellen
Als Basis der Analyse dient die Liste S. Diese enthält alle möglichen Konstellationen der Spielfiguren auf dem Spielbrett.
Hierfür werden zuerst die Figuren mit der Funktion ``place_figure_everywhere_on_every_board`` auf allen Positionen platziert.
Pro Farbe am Zug (schwarz und weiß) werden somit

$$
\frac{64!}{(64-count(pieces))!}
$$

verschiedene Boards erstellt.
Diese werden anschließend mit der Funktion ``check_boards`` auf Validität überprüft und in einer Liste zusammengeführt.

In [52]:
empty_board = chess.Board().empty()
white_boards = [empty_board]
for fig in figures_to_place:
    white_boards = place_figure_everywhere_on_every_board(chess.Piece(fig[0], fig[1]), white_boards)
    #print(len(white_boards))

# All checked boards board.turn = White
S_White = check_boards(white_boards)
#print(len(S_White))

empty_board.turn = chess.BLACK
black_boards = [empty_board]
for fig in figures_to_place:
    black_boards = place_figure_everywhere_on_every_board(chess.Piece(fig[0], fig[1]), black_boards)
    #print(len(black_boards))

# All checked boards board.turn = White
S_Black = check_boards(black_boards)
#print(len(S_Black))

S = S_White + S_Black
print(str(len(S)) + " Boards in S")

399112 Boards in S


## Bestimmung aller $S_n$ Mengen

Als erster Schritt wird die Menge $S_0$ generiert. Diese enthält alle Schachbretter, bei denen die Farbe, die gerade am Zug ist auch im Schachmatt steht. Die anschließenden Schritte beschreiben das Vorgehen zur Bestimmung der Menge $S_0$.

* Über den Inhalt von $S$ iterieren.
* Status-Informationen von der Bibliothek abfragen und überprüfen, ob ein Spieler gewonnen hat.
* Wenn ja, das Objekt zu $S_0$ hinzufügen.
  Weiter werden die Objekte in ihrer String-Representation zu $S_{0-ASCII}$ hinzugefügt.
* Wenn nicht, das Objekt in $S_{tmp}$ hinzufügen. (Dies ist effizienter als das Objekt aus der Liste $S$ zu entfernen)
* Die ursprüngliche Liste S mit $S_{tmp}$ ersetzen.

In [53]:
S_tmp = []
S_0 = []

for board in S:
    outcome = board.outcome()
    # If True Game has finished in a certain way
    if outcome is not None:
        # A winner has been determined
        if outcome.winner is not None and board.is_valid():
            S_0.append(board)
    # Every other board
    else:
        S_tmp.append(board)
S = S_tmp

Der nächste Schritt besteht darin die weiteren $S_{n+1}$ Mengen zu bestimmen. Hierzu werden die bereits bestimmten $S_n$ Mengen genommen und alle Bretter aus S selektiert, die innerhalb eines Zugs in $S_n$ landen. Die Umsetzung erfolgt durch die Funktion ``one_step_away``. Diese erhält unter anderem zwei Listen, mit denen dieses Verfahren durchgeführt werden soll. Alle Funktionsparameter können aus der nachfolgenden Liste entnommen werden:

* ``s_n``: Die Liste mit Board-Objekten, deren Spielbretter sich $n$ Züge vom Sieg entfernt befinden.
* ``s``: Die Liste, aus welcher die nächste Stufe ($n+1$) ausgewählt werden soll.
* ``s_white_tuples``: Menge der ASCII-Representationen aller bereits zugeordneten Spielbretter mit weiß am Zug.
* ``iteration_count``: $m$ des $S_m$, welches gerade berechnet wird.

Der Algorithmus zur Bestimmung der Menge $S_{n+1}$ wird im folgenden Abschnit beschrieben.

* Für effizientere Vergleiche wird für jedes Objekt in $S$ die ASCII-Representation gespeichert.
* Über die Objekte in $S_n$ iterieren.
  * Den Spieler, welcher am Zug ist wechseln (Da, um im aktuellen Zustand anzukommen, der andere Spieler einen Zug
  gemacht hat)
  * Pseudo legale Spielzüge von der Bibliothek berechnen lassen:
    Ein pseudo legaler Spielzug ist ein Spielzug, welcher die grundsätzlichen Bewegungsregeln der Figur einhält,
    aber das Schachbrett unter Umständen in einen nicht regelkonformen Zustand versetzt.
  * Über diese Spielzüge iterieren.
    * Den Zug ausführen.
    * Überprüfen, ob das Board sich in einem erlaubten Zustand befindet.
      Wenn nein, den Schleifendurchlauf abbrechen.
    * Mittels der ASCII-Representation überprüfen, ob das modifizierte Spielbrett in $S$ gefunden wird.
    * Wenn es gefunden wird, die ASCII-Representation zu $s_{n+1-ASCII}$ hinzufügen.
    * Den Zug rückgängig machen.
  * Den Spieler, welcher ursprünglich am Zug war, wiederherstellen.
* Über $S$ iterieren.
  * Wenn sich das Spielbrett in $S_{n+1-ASCII}$ befindet, wird es $S_{n+1}$ hinzugefügt.
    * Besonderheit für Spielbretter, bei welchen Schwarz am Zug ist: Diese werden nur zu $S_{n+1}$ hinzugefügt, wenn
    alle Züge, welche von dieser Position möglich sind, in einem $S_m$ mit $m < n$ enden.
    Für diese Überprüfung wird $S_{white-ASCII}$ verwendet.
  * Wenn sich das Spielbrett nicht in $S_{n+1-ASCII}$ befindet, wird es $S_{tmp}$ hinzugefügt.
* Wurden Positionen mit Weiß am Zug berechnet, diese $S_{white-ASCII}$ hinzufügen.

Außerdem müssen bei der Durchführung des Algorithmus weitere Aspekte berücksichtig werden:

* Da Bauern nur in eine Richtung laufen können, müssen die rückwärts Schritte eines Bauern manuell durchgeführt werden. Bauern werden daher im ersten Schritt ignoriert.
* Bauern, die die oberste Reihe des Spielfeldes erreichen können zu einer anderen Figur eingetauscht werden. Dieser Schritt wird nicht durch die Pseudo-Legal-Moves abgedeckt, daher wird, sollte sich eine Königin in der obersten Reihe befinden, diese manuell durch einen Bauern ersetzt.

Sobald die Funktion ausgeführt wurde, wird ein Tupel bestehend aus den Mengen $S_{n+1}$, $S_{tmp}$ und $S_{white-TUPLE}$ zurückgegeben.

In [54]:
def next_states(s_n, s, s_white_tuples, iteration_count):
    #variables
    s_n1 = []
    s_n1_tmp = []
    s_n1_tuples = set()
    s_tuples = set()
    s_n_tuples = set()
    s_tmp = []

    #create temporary list for comparison
    for chess_board in s:
        s_tuples.add((chess_board.turn, chess_board.__str__()))

    for chess_board in s_n:
        s_n_tuples.add((chess_board.turn, chess_board.__str__()))

    for i in range(len(s_n)):

        if iteration_count is not None:
            status = "Calculating S" + str(iteration_count) + " - Board " + str(i+1) + " of " + str(len(s_n)) + \
                     " from S" + str(iteration_count-1)
        else:
            status = "Board " + str(i+1) + " of " + str(len(s_n))
        clear_output(wait=True)
        print(status)

        # Get current board and invert the player
        chess_board = s_n[i]
        chess_board.turn = chess_board.turn ^ True

        # Find all Pawns
        pawn_at = []
        for pawn in chess_board.pieces(chess.PAWN, True):
            pawn_at.append(chess.square_name(pawn))

        # try moves and check if they remain in S
        for pLMove in chess_board.pseudo_legal_moves:
            if chess.square_name(pLMove.from_square) not in pawn_at:
                chess_board.push(pLMove)
                chess_board.turn = chess_board.turn ^ True
                if not chess_board.is_valid():
                    chess_board.turn = chess_board.turn ^ True
                    chess_board.pop()
                    continue
                # If the new board is found in S, it can be reached in one step
                if (chess_board.turn, chess_board.__str__()) in s_tuples:
                    s_n1_tuples.add((chess_board.turn, chess_board.__str__()))
                    chess_board.turn = chess_board.turn ^ True
                chess_board.pop()
        '''
        * As pawns only move forward their move cannot cannot be reversed through the pseudo legal moves
        * to mitigate this the pawn is moved back one row manually
        '''
        for pawn in chess_board.pieces(chess.PAWN, True):
            if chess_board.piece_at(pawn - 9) is None:
                chess_board.remove_piece_at(pawn)
                chess_board.set_piece_at(pawn - 9, chess.Piece.from_symbol('P'))
                if chess_board.is_valid():
                    if (chess_board.turn ^ True, chess_board.__str__()) in s_tuples:
                        s_n1_tuples.add((chess_board.turn ^ True, chess_board.__str__()))
                chess_board.remove_piece_at(pawn - 9)
                chess_board.set_piece_at(pawn, chess.Piece.from_symbol('P'))
        '''
        * As pawns can be exchanged to other figures and this cannot be undone this exchange can also not be reversed
        * through the pseudo legal moves.
        * To mitigate this a pawn will be placed in the top row in the spot of a queen
        '''
        queen = check_top_row(chess_board)
        if queen:
            for square in queen:
                chess_board.remove_piece_at(square)
                chess_board.set_piece_at(square, chess.Piece.from_symbol('P'))
                if chess_board.is_valid():
                    if (chess_board.turn ^ True, chess_board.__str__()) in s_tuples:
                        s_n1_tuples.add((chess_board.turn ^ True, chess_board.__str__()))
                chess_board.remove_piece_at(square)
                chess_board.set_piece_at(square, chess.Piece.from_symbol('Q'))

        # Restore the original state of the board
        chess_board.turn = chess_board.turn ^ True

    clear_output(wait=True)
    if iteration_count is not None:
        print("Calculating S" + str(iteration_count) + " - Mapping ASCII boards to board objects")
    else:
        print("Mapping ASCII boards to board objects")
    #create return lists
    for chess_board in s:
        if(chess_board.turn, chess_board.__str__()) in s_n1_tuples:
            s_n1.append(chess_board)
        else:
            s_tmp.append(chess_board)


    clear_output(wait=True)
    if iteration_count is not None:
        print("Calculating S" + str(iteration_count) + " - Checking Black Moves for determinism")
    else:
        print("Checking Black Moves for determinism")

    '''
    * We can choose every turn white does, but not for black.
    * If one particular move from black brings the board from S_n to S_n-1 it is not guaranteed that every move
    * black can do does so as well.
    * To mitigate this only boards where every move ends up in a lower denomination of S will be added to the currently
    * calculated one.
    * For efficiency an ascii representation is carried through all calculations.
    '''
    # Only needed for Black-Moves
    for chess_board in s_n1:
        include = True
        if not chess_board.turn:
            for move in chess_board.legal_moves:
                chess_board.push(move)
                str_rep = (chess_board.turn, chess_board.__str__())
                chess_board.pop()
                if str_rep not in s_white_tuples:
                    include = False
        if include:
            s_n1_tmp.append(chess_board)
        else:
            s_tmp.append(chess_board)
    s_n1 = s_n1_tmp

    # Only needed for White-Moves
    if iteration_count % 2 == 1:
        s_white_tuples = s_white_tuples | s_n1_tuples


    clear_output(wait=True)
    print("Done with S" + str(iteration_count) + " - " + str(len(s_tmp)) + " Boards remaining in S")



    return s_n1, s_tmp, s_white_tuples

Ein Problem, das bei der Verwendung der rückwärts-Analyse auftritt, liegt in dem Szenario: "König und Bauer gegen König". Dieses Szenario beinhaltet nämlich die Umwandlung des Bauerns in eine andere Figur (Dame, Turm, Läufer, Springer). Da in den meisten Fällen die Umwandlung in eine Dame am sinnvollsten ist, wurde eine Funktion geschrieben, die potentielle Umwandlungen ausfindig machen soll.

Bei dieser Funktion handelt es sich um die Funktion ``check_top_row``. Diese erhält als Parameter ein ``board``. Bei diesem Spielbrett wird für die oberste Reihe (Sichtweise weiß, da diese Seite nur eine Dame in den Szenario erhalten kann) jedes Feld überprüft und jedes Feld zurückgegeben, auf dem sich eine Dame befindet.

In [55]:
def check_top_row(board):
    return_list = []
    for i in range(56, 64):
        if board.piece_type_at(i) == chess.QUEEN:
            return_list.append(i)

    if len(return_list) > 0:
        return return_list
    else:
        return False

Mit den zuvor definierten Funktionen ist es nun möglich alle $S_n$ zu bestimmen. Hierfür wird die Funktion ``one_step_away`` solange aufgerufen, bis sich die Menge $S$ nicht mehr verändert (Kann auch leer sein).

In [56]:
S_n_sequence = [S_0]
S_White_Tuples = set()

while True:
    S_n_new, S, S_White_Tuples = next_states(S_n_sequence[-1], S, S_White_Tuples, len(S_n_sequence))
    if not S_n_new: #an empty list is false
        break
    S_n_sequence.append(S_n_new)

print("Done")
print(str(len(S_n_sequence)) + " S-Lists calculated")
print(str(len(S)) + " Boards could not be matched into an n.")

Done with S33 - 22176 Boards remaining in S
Done
33 S-Lists calculated
22176 Boards could not be matched into an n.


Nach erfolgreicher Berechnung werden die FENs der Boards in eine Datei geschrieben, um eine spätere Auswertung zu ermöglichen.

In [57]:
S_n_seq_ascii = []
S_ascii = []

for i in range(len(S_n_sequence)):
    S_n_ascii = []
    for board in S_n_sequence[i]:
        S_n_ascii.append(board.fen())
    S_n_seq_ascii.append(S_n_ascii)

for board in S:
    S_ascii.append(board.fen())

f = open("S_n_seq.json", "w")
f.write(json.dumps(S_n_seq_ascii))
f.close()

f = open("S_rem.json", "w")
f.write(json.dumps(S_ascii))
f.close()